In [1]:
import joblib

model=joblib.load('brain_tumor_segmentation.sav')

In [2]:
clusterNumber=4

In [3]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
imgs_names=os.listdir('images')

for img_name in imgs_names:
    img_path=os.path.join('images',img_name)
    original_img=cv2.imread(img_path)
    img=cv2.imread(img_path,0)
    height,width=img.shape
    flattenImg=img.reshape(height*width,1)
    
    predicted_labels=model.predict(flattenImg)
    labelsToArray=predicted_labels.reshape(height,width)
    
    mask=(labelsToArray==clusterNumber)
    
    extracted_tumor=np.array((mask*255),dtype=np.uint8)
    kernel=np.ones((5,5),np.uint8)
    extracted_tumor=cv2.erode(extracted_tumor,kernel,iterations=1)
    contours,hierarchy=cv2.findContours(extracted_tumor,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        area=cv2.contourArea(contour)
        if(area>1000):
            cv2.drawContours(original_img,[contour],-1,(0,255,255),3)
            x,y,w,h = cv2.boundingRect(contour)
            cv2.rectangle(original_img,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.rectangle(original_img,(x,y),(x+w,y-40),(0,255,0),-1)
            cv2.putText(original_img,"tumor",(x+10,y-30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    cv2.imshow('WINDOW',original_img)
    cv2.waitKey(2000)
    cv2.destroyAllWindows()
    
    